# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
RANDOM_STATE = 12345

In [3]:
data = pd.read_csv('datasets/insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Колонки 'Возраст' и 'Зарплата' имеют тип данных float64. Посмотрим, есть ли в них дроби

In [4]:
display(data['Возраст'].unique())
display(data['Зарплата'].unique())

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

array([49600., 38000., 21000., 41700., 26100., 41000., 39700., 38600.,
       49700., 51700., 36600., 29300., 39500., 55000., 43700., 23300.,
       48900., 33200., 36900., 43500., 36100., 26600., 48700., 40400.,
       38400., 34600., 34800., 36800., 42200., 46300., 30300., 51000.,
       28100., 64800., 30400., 45300., 38300., 49500., 19400., 40200.,
       31700., 69200., 33100., 31600., 34500., 38700., 39600., 42400.,
       34900., 30500., 24200., 49900., 14300., 47000., 44800., 43800.,
       42700., 35400., 57200., 29600., 37400., 48100., 33700., 61800.,
       39400., 15600., 52600., 37600., 52500., 32700., 51600., 60900.,
       41800., 47400., 26500., 45900., 35700., 34300., 26700., 25700.,
       33300., 31100., 31500., 42100., 37300., 42500., 27300., 46800.,
       33500., 44300., 41600., 53900., 40100., 44600., 45000., 32000.,
       38200., 33000., 38500., 51800., 33800., 46400., 43200., 31800.,
       50200., 35100., 30700., 45800., 49300., 42800., 33600., 50300.,
      

Дробей нет, значит  можно привести к типу данных int64

In [5]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int32
 2   Зарплата           5000 non-null   int32
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int32(2), int64(3)
memory usage: 156.4 KB


In [6]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [7]:
data.duplicated().sum()

153

In [8]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int32
 2   Зарплата           4847 non-null   int32
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int32(2), int64(3)
memory usage: 189.3 KB


**Вывод**
- Пропусков нет
- Приведены типы данных
- Удалены явные дубликаты
- Данные готовы

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ: Качество не  изменится** 

**Обоснование:** 

Составим формулу предсказаний $a_1$ для матрицы признаков $X$ умноженную на обратимую матрицу $P$ (то есть $XP$) 

$ a = X(X^T X)^{-1} X^T y $

$a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y $

Для раскрытия скобок можно воспользоваться одним из свойств транспонированных матриц: 

Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке.
$$
(AB)^T = B^T A^T 
$$

$a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y = XP (P^T X^T XP)^{-1} (XP)^T y$

Теперь нужно вынести множители из-под знака обратной матрицы. Для этого применим правило:

Обратная матрица от произведения двух, в нащем случае трёх матриц равна произведению обратных матриц взятых в обратном порядке.

$$
(ABC)^{-1} = C^{-1} B^{-1} A^{-1} 
$$

Это свойство применимо только для квадратных матриц. В нашем случае квадратные матрицы $P^T$, $(X^TX)$ и $P$.

$a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y = XP (P^T X^T XP)^{-1} (XP)^T y = XPP^{-1}(X^T X)^{-1}(P^T)^{-1}P^T X^T y $

$PP^{-1} = E$

$(P^T)^{-1}P^T = E_1$

$E$ , $E_1$ - Единичные матрицы

Свойство квадратной матрицы:

$AE = EA = A$

$a_1 = XE(X^T X)^{-1}E_1 X^T y = X(X^T X)^{-1} X^T y = a$

Выражения тождественны

## Алгоритм преобразования

**Алгоритм**

Умножение матрицы признаков на обратимую матрицу, генерируюмую случайно.

Сгенерированная  матрица должна быть той же размерности, что и матрица признаков

Этапы алгоритма:
1. Проверка r2 на изначальных данных
2. Генерируется случайная матрица
3. Сгенерированная матрица проверяется на обратимость
4. Производится матричное умножение матрицы признаков на сгенерированную матрицу (шифрование)
5. Вычисляется r2 с зашифрованными признаками
6. Сравнение r2
7. Проверка дешифровки

**Обоснование**

Качество линейной регрессии не поменяется, так как произведение матрицы состоит из всех возможных комбинаций скалярных произведений вектор-строк матрицы с признаками и вектор-столбцов случайно обратимой матрицы. Таким образом мы произведем умножение каждого набора признаков на одинаковые наборы векторов и запишем их в новый вектор - строку. Соответственно итоговые веса каждого такого вектора будут примерно соотносится с весами набора признаков для каждого нового набора векторов в результирующей матрице.

## Проверка алгоритма

Разделим данные выборки и посмотрим на r2

In [10]:
features = data.drop("Страховые выплаты", axis=1)
target = data["Страховые выплаты"]
n = features.shape[1]

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)

model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(r2_score(target_test, predictions))

0.42307727615837565


Создание матрицы

In [11]:
random_2 = []
for i in range (50):
    try:
          i = np.random.randint(0,5, size=(n,n))
          k = np.linalg.inv(i)
          one = np.eye(n)
          m = i @ k
          if m[3].tolist() == one[3].tolist():
              if m[2].tolist() == one[2].tolist():
                  if m[1].tolist() == one[1].tolist():
                      if m[0].tolist() == one[0].tolist():
                          random_2.append(i)
    except: print('Вы́рожденная ма́трица')

Вы́рожденная ма́трица
Вы́рожденная ма́трица


In [12]:
random_matrix = random_2[0]
inverse = np.linalg.inv(random_matrix)
random_matrix @ inverse

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

Умножение на матрицу признаков

In [13]:
new_features = features.values @ random_matrix
display(new_features)

array([[198570, 148882, 148885,      7],
       [152188, 114092, 114094,      4],
       [ 84116,  63058,  63058,      0],
       ...,
       [135688, 101740, 101744,      8],
       [130902,  98144,  98151,     15],
       [162518, 121856, 121859,      7]], dtype=int64)

Смотрим r2 на новых признаках

In [14]:
new_features_train, new_features_test, new_target_train, new_target_test = train_test_split(
    new_features, target, test_size=0.25, random_state=RANDOM_STATE)

model = LinearRegression()
model.fit(new_features_train, new_target_train)
new_predicitions = model.predict(new_features_test)
print(r2_score(new_target_test, new_predicitions))

0.42307727615852664


r2 не изменился

Теперь дешифруем данные

In [15]:
original = new_features_test @ np.linalg.inv(random_matrix)
a = pd.DataFrame(new_target_test)
b = pd.DataFrame(original, index = a.index, columns=features.columns).join(a)
c = pd.DataFrame(predictions, index = a.index)
predict = []
for i in c[0]:
    if i > 0.5:
        predict.append('1')
    else:
        predict.append('0')
b['Предсказания'] = predict

In [16]:
b.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты,Предсказания
1335,1.0,23.0,47200.0,0.0,0,0
3875,1.0,32.0,65900.0,0.0,0,0
166,1.0,33.0,33900.0,2.0,0,0
3003,0.0,39.0,19300.0,2.0,0,0
424,0.0,46.0,27500.0,2.0,1,1


## Вывод

Были предоставлены данные клиентов страховой компании «Хоть потоп».

Поставлена задача разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Данные, необходимо преобразовать так, чтобы после преобразования качество моделей машинного обучения не ухудшилось.

Предоставленные данные были изучены и подготовлены. При подготовке из данных были удалены явные дубликты и провелось приведение к новому типу данных. Пропусков и других аномалий в предоставленных данных нет.

Перед разработкой метода преобразования встал вопрос, изменится ли качество модели линейной регрессии, если матрицу признаков умножают на обратимую матрицу. Ответ: качество не изменится. Значит можно использовать данный метод для преобразоваания данных.

Алгоритм преобразования заключается в следующем:

Матрица признаков умножается на обратимую матрицу, которая генерируются случайно, при этом такая матрица должна быть той же размерности, что и матрица признаков. 

Этапы алгоритма:
1. Проверка r2 на изначальных данных
2. Генерируется случайная матрица
3. Сгенерированная матрица проверяется на обратимость
4. Производится матричное умножение матрицы признаков на сгенерированную матрицу (шифрование)
5. Вычисляется r2 с зашифрованными признаками
6. Сравнение r2
7. Проверка дешифровки

После применения данного алгоритма проведена проверка качества модели до и после преобразования. Качество не изменилось.

Таким образом, при применении данного метода можно зашифровать данные клиентов страховой компании «Хоть потоп».

Поставленная задача выполнена